In [1]:
from pandas_profiling import ProfileReport
import matplotlib
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

np.random.seed(1001)

df = pd.read_csv('chronic_kidney_disease_full.csv', na_values = '?', encoding='latin-1')
print ("df Shape: ", df.shape)

df Shape:  (400, 25)


In [2]:
# Let's have a look at the data and check whether there are missing values
ProfileReport(df)

/Users/ozancaneren/opt/anaconda3/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,25
Number of observations,400
Total Missing (%),10.1%
Total size in memory,78.2 KiB
Average record size in memory,200.3 B
Numeric,14
Categorical,11
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [12]:
#  Based on the data profile above, let's fill out the NA values with either the mean or the mod of
#  the corresponding column. The missing values in the columns "rbc" and "pc" are filled based on the frequency
#  of the values in them.

df['age'].fillna(round(df.age.mean()), inplace = True)
df['bp'].fillna(round(df.bp.mean(),-1), inplace = True)
df['sg'].fillna(1.015, inplace = True)
df['al'].fillna(round(df.al.mean()), inplace = True)
df['su'].fillna(0, inplace = True)
df['rbc'] = df['rbc'].astype(object).fillna(pd.Series(np.random.choice(['normal', 'abnormal'], p=[201/248, 47/248], size=len(df))))
df['pc'] = df['pc'].astype(object).fillna(pd.Series(np.random.choice(['normal', 'abnormal'], p=[259/335, 76/335], size=len(df))))
df['pcc'] = df['pcc'].astype(object).fillna('notpresent')
df['ba'] = df['ba'].astype(object).fillna('notpresent')
df['bgr'].fillna(round(df.bgr.mean()), inplace = True)
df['al'].fillna(round(df.al.mean()), inplace = True)
df['bu'].fillna(round(df.bu.mean()), inplace = True)
df['sc'].fillna(round(df.sc.mean(),1), inplace = True)
df['sod'].fillna(round(df.sod.mean()), inplace = True)
df['pot'].fillna(round(df.pot.mean(),1), inplace = True)
df['hemo'].fillna(round(df.hemo.mean(),1), inplace = True)
df['pcv'].fillna(round(df.pcv.mean()), inplace = True)
df['wbcc'].fillna(round(df.wbcc.mean(),-2), inplace = True)
df['rbcc'].fillna(round(df.rbcc.mean(),1), inplace = True)
df['htn'] = df['htn'].astype(object).fillna('no')
df.dm = df.dm.replace({' yes':'yes', '\tyes':'yes', '\tno':'no'})
df['dm'] = df['dm'].astype(object).fillna('no')
df.cad=df.cad.replace({'\tno':'no'})
df['cad'] = df['cad'].astype(object).fillna('no')
df['appet'] = df['appet'].astype(object).fillna('good')
df['pe'] = df['pe'].astype(object).fillna('no')
df['ane'] = df['ane'].astype(object).fillna('no')

df = df.sample(frac=1)
labels = df['class']
df = df.drop('class', axis = 1)
df = pd.get_dummies(df)
print ("df Shape: ", df.shape)

df Shape:  (400, 34)


In [13]:
# Let's split the data into train and test
X_Train, X_Test, Y_Train, Y_Test = train_test_split(df, labels, test_size = 0.2, random_state = 8)

print ("Train_x Shape: ", X_Train.shape)
print ("Train_y Shape: ", Y_Train.shape)
print ("Test_x Shape: ", X_Test.shape)
print ("Test_y Shape: ", Y_Test.shape)
df

Train_x Shape:  (320, 34)
Train_y Shape:  (320,)
Test_x Shape:  (80, 34)
Test_y Shape:  (80,)


,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,dm_no,dm_yes,cad_no,cad_yes,appet_good,appet_poor,pe_no,pe_yes,ane_no,ane_yes
305,41.0,80.0,1.020,0.0,0.0,122.0,25.0,0.8,138.0,5.0,...,1,0,1,0,1,0,1,0,1,0
96,60.0,80.0,1.010,1.0,1.0,172.0,32.0,2.7,138.0,4.6,...,0,1,0,1,0,1,1,0,1,0
299,73.0,60.0,1.020,0.0,0.0,127.0,48.0,0.5,150.0,3.5,...,1,0,1,0,1,0,1,0,1,0
189,64.0,60.0,1.010,4.0,1.0,239.0,58.0,4.3,137.0,5.4,...,0,1,1,0,0,1,0,1,1,0
367,68.0,60.0,1.025,0.0,0.0,125.0,41.0,1.1,139.0,3.8,...,1,0,1,0,1,0,1,0,1,0
387,15.0,80.0,1.025,0.0,0.0,93.0,17.0,0.9,136.0,3.9,...,1,0,1,0,1,0,1,0,1,0
307,47.0,60.0,1.020,0.0,0.0,137.0,17.0,0.5,150.0,3.5,...,1,0,1,0,1,0,1,0,1,0
153,55.0,90.0,1.010,2.0,1.0,273.0,235.0,14.2,132.0,3.4,...,0,1,1,0,0,1,0,1,0,1
68,65.0,70.0,1.010,2.0,0.0,112.0,73.0,3.3,138.0,4.6,...,1,0,1,0,1,0,1,0,1,0
190,6.0,60.0,1.010,4.0,0.0,94.0,67.0,1.0,135.0,4.9,...,1,0,1,0,0,1,1,0,1,0


In [14]:
# Now, we can start fitting models

clf = DecisionTreeClassifier(random_state=44)
bgc = BaggingClassifier(base_estimator=clf,random_state=44)
rfc = RandomForestClassifier(random_state=44)
abc = AdaBoostClassifier(random_state=44)
sgb = GradientBoostingClassifier(random_state=44)
xgb = XGBClassifier(objective='binary:logistic', random_state=44)
lgbm = lgb.LGBMClassifier(random_state=44)
cb = CatBoostClassifier(verbose=False, random_state=44)

clf_cv_score = cross_val_score(clf, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - DTR: ", clf_cv_score.mean())

bgc_cv_score = cross_val_score(bgc, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - BAG: ", bgc_cv_score.mean())

rfc_cv_score = cross_val_score(rfc, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - RFC: ", rfc_cv_score.mean())

abc_cv_score = cross_val_score(abc, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - ADA: ", abc_cv_score.mean())

sgb_cv_score = cross_val_score(sgb, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - GBC: ", sgb_cv_score.mean())

xgb_cv_score = cross_val_score(xgb, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - XGB: ", xgb_cv_score.mean())

lgbm_cv_score = cross_val_score(lgbm, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - LGBM: ", lgbm_cv_score.mean())

cb_cv_score = cross_val_score(cb, X_Train, Y_Train, cv=4, scoring='accuracy')
print("Mean Accuracy Score - CB: ", lgbm_cv_score.mean())

Mean Accuracy Score - DTR:  0.9500000000000001
Mean Accuracy Score - BAG:  0.9624999999999999
Mean Accuracy Score - RFC:  0.9781250000000001
Mean Accuracy Score - ADA:  0.975
Mean Accuracy Score - GBC:  0.9656250000000001
Mean Accuracy Score - XGB:  0.971875
Mean Accuracy Score - LGBM:  0.978125
Mean Accuracy Score - CB:  0.978125


In [311]:
# It can be seen that the highest Cross Validation Accuracy is achieved by using the Gradient Boosting Classifier
# Let's see the prediction accuracy score of Gradient Boosting Classifier on the Test Data

rfc = RandomForestClassifier(random_state=44)

rfc = rfc.fit(X_Train, Y_Train)

#Predict the response for the test dataset
Y_Pred = rfc.predict(X_Test)

print(confusion_matrix(Y_Test, Y_Pred))
print ("GBC Accuracy Score On the Test Data:", accuracy_score(Y_Test, Y_Pred))
acc1 = accuracy_score(Y_Test, Y_Pred)

[[48  1]
 [ 2 29]]
GBC Accuracy Score On the Test Data: 0.9625


In [312]:
# Let's try tuning the hyperparameters of Gradient Boosting by using a Randomized Search so that we 
# get a higher accuracy score on the test data

random_search = {'criterion': ['entropy', 'gini'],
                 'max_depth': list(np.linspace(10, 50, 5, dtype = int)) + [None],
                 'max_features': ['auto', 'sqrt','log2', None],
                 'min_samples_leaf': [2, 3, 4, 5, 6],
                 'min_samples_split': [3, 4, 5, 6],
                 'n_estimators': list(np.linspace(100, 150, 6, dtype = int))}

rfc = RandomForestClassifier(random_state=44)
model = RandomizedSearchCV(estimator = rfc, param_distributions=random_search, scoring='accuracy', n_iter=20, 
                               cv = 4, verbose=0, random_state=44, n_jobs = -1)
model.fit(X_Train,Y_Train)

print("Random Search")
prediction1 = model.best_estimator_.predict(X_Test)
print(confusion_matrix(Y_Test, prediction1))
print("RFC Accuracy Score On the Test Data (Tuned with Random Search):", metrics.accuracy_score(Y_Test,prediction1))
acc2 = accuracy_score(Y_Test, prediction1)
model.best_params_

Random Search
[[49  0]
 [ 2 29]]
RFC Accuracy Score On the Test Data (Tuned with Random Search): 0.975


{'n_estimators': 120,
 'min_samples_split': 4,
 'min_samples_leaf': 6,
 'max_features': 'sqrt',
 'max_depth': 50,
 'criterion': 'entropy'}

In [313]:
# Let's try fine tuning the hyperparameters of Gradient Boosting even more by using a Grid Search so we can achieve
# an accuracy score that is even higher than the one we got by Hyperparameter Tuning with Random Search above

grid_search = { 'criterion': [model.best_params_['criterion']],
                'max_depth': [model.best_params_['max_depth']],
                'max_features': [model.best_params_['max_features']],
                'min_samples_leaf': [model.best_params_['min_samples_leaf'] - 1, 
                                     model.best_params_['min_samples_leaf'], 
                                     model.best_params_['min_samples_leaf'] + 1],
                'min_samples_split': [model.best_params_['min_samples_split'] - 1, 
                                      model.best_params_['min_samples_split'], 
                                      model.best_params_['min_samples_split'] + 1],
                'n_estimators': [model.best_params_['n_estimators'] - 5, 
                                 model.best_params_['n_estimators'], 
                                 model.best_params_['n_estimators'] + 5],}

rfc = RandomForestClassifier(random_state=44)
model = GridSearchCV(estimator = rfc, param_grid = grid_search, scoring='accuracy', 
                               cv = 4, verbose=0, n_jobs = -1)
model.fit(X_Train,Y_Train)


print("Grid Search")
prediction2 = model.best_estimator_.predict(X_Test)
print(confusion_matrix(Y_Test, prediction2))
print("RFC Accuracy Score On the Test Data (Tuned with Grid Search):", metrics.accuracy_score(Y_Test,prediction2))
acc3 = accuracy_score(Y_Test, prediction2)
model.best_params_

Grid Search
[[49  0]
 [ 2 29]]
RFC Accuracy Score On the Test Data (Tuned with Grid Search): 0.975


{'criterion': 'entropy',
 'max_depth': 50,
 'max_features': 'sqrt',
 'min_samples_leaf': 6,
 'min_samples_split': 3,
 'n_estimators': 120}

In [314]:
#   The accuracy improvements on the base model (Gradient Boosting Classifier) achieved by using
#   "Random Search" and "Grid Search" are as follows

print('Base Accuracy vs Random Search {:0.4f}%.'.format( 100 * (acc2 - acc1) / acc1))
print('Base Accuracy vs Grid Search {:0.4f}%.'.format( 100 * (acc3 - acc1) / acc1))

Base Accuracy vs Random Search 1.2987%.
Base Accuracy vs Grid Search 1.2987%.
